In [2]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot="True")

Extracting MNIST_data/train-images-idx3-ubyte.gz


/home/rahul/anaconda2/envs/tensorflow/lib/python2.7/gzip.py:275: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
input_data.py:55: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [1]:
import tensorflow as tf

In [4]:
x  = tf.placeholder("float", shape=[None, 784]) #feed in image pixels laid out in a line
y_true = tf.placeholder("float", shape=[None, 10]) #True labels here

In [5]:
x_image = tf.reshape(x, [-1, 28, 28, 1]) #Egs,W,H,C
# -1 can also be used to infer the shape

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return(tf.Variable(initial))

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return(tf.Variable(initial))

#Variables can be first intialized with some variables as constants
#Then converted to tf.Variable

In [7]:
def conv2d(x, W):
    ans = tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
    return(ans)

def max_pool_2x2(x):
    ans = tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    return(ans)
    

In [8]:
#1st CONV init
# 32 filters each with 5x5 weights with 1 channel
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

In [9]:
#1st CONV calc
actv1 = conv2d(x_image, W_conv1) + b_conv1
h_conv1 = tf.nn.relu(actv1)
h_pool1 = max_pool_2x2(h_conv1)

In [10]:
#2nd CONV init
W_conv2 = weight_variable([5,5,32,64]) #64 filters instead
#Now 32 channels because 32 incoming filters aka layers
b_conv2 = bias_variable([64])

In [11]:
#2nd CONV calc
actv2 = conv2d(h_pool1, W_conv2) + b_conv2
h_conv2 = tf.nn.relu(actv2)
h_pool2 = max_pool_2x2(h_conv2)

In [12]:
#Final FC init
#Input = 7*7*64, output is 1024. Then we shall get it down to 10
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

In [13]:
flattened_filters = tf.reshape(h_pool2, [-1, 7*7*64])
actvfc1 = tf.matmul(flattened_filters, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(actvfc1)

In [14]:
keep_prob = tf.placeholder("float") # % dropout to keep
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


In [15]:
#Final softmax to classify init

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

In [16]:
#Calc
actvfc2 = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
y_pred = tf.nn.softmax(actvfc2)

In [17]:
cross_entropy = - tf.reduce_sum(y_true*tf.log(y_pred))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_pred,1) , tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [ ]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())

for iter in xrange(200):
    batch = mnist.train.next_batch(50)
    
    if iter%100==0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_true:batch[1], keep_prob:1.0 })
        print("Step %d, train acc %g" % (iter, train_accuracy))
        
    sess.run(train_step, feed_dict={x:batch[0], y_true:batch[1], keep_prob:0.5})
    
test_accuracy = sess.run(accuracy, feed_dict={x : mnist.test.images, y_true : mnist.test.labels, keep_prob : 1.0})
print("Test acc %g" % test_accuracy)

In [ ]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
import tensorflow as tf

x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

x_image = tf.reshape(x, [-1,28,28,1])
print "x_image="
print x_image

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.Session()

sess.run(tf.initialize_all_variables())

for i in range(200):
   batch = mnist.train.next_batch(50)
   if i%10 == 0:
     train_accuracy = sess.run( accuracy, feed_dict={ x:batch[0], y_: batch[1], keep_prob: 1.0})
     print("step %d, training accuracy %g"%(i, train_accuracy))
   sess.run(train_step,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"% sess.run(accuracy, feed_dict={ 
       x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Extracting MNIST_data/train-images-idx3-ubyte.gz


/home/rahul/anaconda2/envs/tensorflow/lib/python2.7/gzip.py:275: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
input_data.py:55: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
x_image=
Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
